In [2]:
import requests
import json
import pandas as pd

NOTION_API_KEY = "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"
NOTION_URL = "https://api.notion.com/v1/pages"
PAGE_ID = "15b2ec8473e98007885fe7ec343ff6df"

headers = {
    "Authorization": "Bearer " + NOTION_API_KEY,
    "Content-Type": "application/json", 
    "Notion-Version": "2022-06-28", 
}

In [ ]:
#create
def data_conversion_4_adding_row(data: dict):
    response = {}
    for x in data:
        if data[x]["type"] == "title":
            tmp_data = {"title" : [{"text": {"content": data[x]["value"]}}]}
        elif data[x]["type"] == "number":
            tmp_data = {"number": data[x]["value"]}
        elif data[x]["type"] == "rich_text":
            tmp_data = {"rich_text": [{"text": {"content": data[x]["value"]}}]}
        elif data[x]["type"] == "date":
            tmp_data = {"date": {"start": data[x]["value"], "end": None}}
        elif data[x]["type"] == "url":
            tmp_data = {"url": data[x]["value"]}
        response.update({x: tmp_data})

    return {"properties": response}

def data_conversion_4_create(data: dict):
    response = {}
    has_title = False
    for x in data:
        if data[x]["type"] == "title":
            tmp_data = {"title" : {}}
            has_title = True

        elif data[x]["type"] == "number":
            tmp_data = {"number": {}}

        elif data[x]["type"] == "rich_text":
            tmp_data = {"rich_text": {}}

        elif data[x]["type"] == "date":
            tmp_data = {"date": {}}

        elif data[x]["type"] == "url":
            tmp_data = {"url": {}}         

        response.update({x: tmp_data})
    
    if not has_title:
        response.update({"Title": {"title": {}}})
    return {"properties": response}

def add_data(id:str, data: dict):
    payload = {
        "parent" : {"database_id" : id} 
    }
    payload.update(data_conversion_4_adding_row(data))
    response = requests.post("https://api.notion.com/v1/pages", headers=headers, json=payload)

    if response.status_code == 400:
        print("Failed to add row")
        return response.json()
    
    print("Row added successfully")
    return response.json()

def create_customer(id:str, data: dict):
    payload = {
        "parent" : {"type": "page_id", "page_id": PAGE_ID},
        "title": [{
            "type": "text",
            "text": {"content": id}
        }]
    }
    payload.update(data_conversion_4_create(data))
    response = requests.post("https://api.notion.com/v1/databases", headers=headers, json=payload)

    if response.status_code == 400:
        print("Failed to create customer")
        return response.json()
    
    print("Customer created successfully")

    add_data(response.json()["id"], data)

    return response.json()


In [ ]:
#retrieve
def read_db():
    url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
    response = requests.get(url, headers=headers)

    data = response.json()
    result = data["results"]

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.get(next_url, headers=headers)
        data = response.json()  
        result += data["results"]

    if result == []: 
        return None
    return result

def display_db():
    res = read_db()
    if res == None:
        return None
    
    list_of_data = []

    for i in range(len(res)):
        list_of_data.append(res[i]["child_database"]["title"])

    return list_of_data

def display_customer_data(dbID : str):
    url = f"https://api.notion.com/v1/databases/{dbID}/query"
    response = requests.post(url, headers=headers, json={"page_size": 100})

    data = response.json()
    result = data["results"]

    if result == []: 
        return None

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.post(next_url, headers=headers)
        data = response.json()  
        result += data["results"]
    
    list_of_data = {}

    for x in result[0]["properties"]:
        list_of_data[x] = []

    for x in result:
        for y in x["properties"]:
            if x["properties"][y]["type"] == "title":
                list_of_data[y].append(x["properties"][y]["title"][0]["text"]["content"])
            elif x["properties"][y]["type"] == "number":
                list_of_data[y].append(x["properties"][y]["number"])
            elif x["properties"][y]["type"] == "rich_text":
                list_of_data[y].append(x["properties"][y]["rich_text"][0]["text"]["content"])
            elif x["properties"][y]["type"] == "date":
                list_of_data[y].append(x["properties"][y]["date"]["start"])
            elif x["properties"][y]["type"] == "url":
                list_of_data[y].append(x["properties"][y]["url"])

    return list_of_data

def get_customer_data(customer_ID : str):
    res = read_db()
    if res == None:
        return None
    
    for i in range(len(res)):
        if res[i]["child_database"]["title"] == customer_ID:
            return display_customer_data(res[i]["id"])
    
    return None

def get_customer_data_id(customer_ID : str):
    res = read_db()
    if res == None:
        return None
    
    for i in range(len(res)):
        if res[i]["child_database"]["title"] == customer_ID:
            return res[i]["id"]

get_customer_data("fast")

In [ ]:
'''def update_transaction(pageID, ID=None, time=None, amount=None, category=None, description=None):
    update_url = f"https://api.notion.com/v1/pages/{pageID}"
    data = {}
    if ID is not None:
        data["ID"] = {"title" : [{"text" : {"content" : str(ID)}}]}
    if time is not None:
        data["Time"] = {"date" : {"start" : time, "end" : None}}
    if amount is not None:
        data["Amount"] = {"number" : amount}
    if category is not None:
        list_of_categories = convert_category_to_list(category)
        data["Category"] = {"multi_select" : list_of_categories}
    if description is not None:
        data["Description"] = {"rich_text" : [{"text" : {"content" : description}}]}

    payload = {"properties": data}
    response = requests.patch(update_url, headers=headers, json=payload)
    print(response.status_code)
    return response


def delete_transaction(pageID : str):
    url = f"https://api.notion.com/v1/pages/{pageID}"

    payload = {"archived": True}

    response = requests.patch(url, headers=headers, json=payload)
    print(response.status_code)
    return response
    
https://www.notion.so/1652ec8473e9815d997aeae883c2d554?v=1652ec8473e9817392e0000ca593275e&pvs=4
    '''

In [ ]:
#update
def changeID(new_name: str, dbID: str):
    payload = {
        "title": [
            {
                "type": "text",
                "text": {
                    "content": new_name
                }
            }
        ]
    }

    url = f"https://api.notion.com/v1/pages/{dbID}"
    response = requests.patch(url, headers=headers, json=payload)

    return response.json()


def update_customer_db(id : str, data: dict):
    url = f"https://api.notion.com/v1/pages/{id}"
    response = requests.patch(url, headers=headers, json=data_conversion_4_adding_row(data))

    if response.status_code == 400:
        print("Failed to update customer")
        return response.json()
    
    print("Customer updated successfully")

    return response.json()

def add_column(id: str, data: dict):
    pass

def delete_column(id: str, data: dict):
    pass

In [ ]:
sample_data = {
    "Name" : {
        "type": "title",
        "value" : "con khi hip hop"
    },
    "age" : {
        "type": "number",
        "value": 25
    },
    "Date" : {
        "type": "date",
        "value": "1997-01-01"
    },
    "fb" : {
        "type": "url",
        "value": "https://www.hiphop.vn"
    },
    "description" : {
        "type": "rich_text",
        "value": "sieu cap hip hop"
    }
}

update_customer_db("15b2ec8473e9800ebac5e2e551a7727d", sample_data)

In [ ]:
#delete
def delete_customer_db(id : str):
    url = f"https://api.notion.com/v1/pages/{id}"
    response = requests.delete(url, headers=headers)

    return response.json()

def delete_data(row_id: str):
    url = f"https://api.notion.com/v1/pages/{row_id}"

    response = requests.patch(url, headers=headers, json={"archived": True})

    print(response.status_code)
    return response

In [3]:
requests.patch("https://api.notion.com/v1/databases/1652ec84-73e9-815d-997a-eae883c2d554", headers=headers, json={"archived": False})

<Response [200]>